In [ ]:
import numpy as np
from numpy import sin, cos,exp,pi
from time import process_time, time
from scipy.optimize import curve_fit, least_squares
from scipy.fft import fft,fftshift,fft2, fftfreq, ifft
from scipy.signal import oaconvolve, medfilt2d

from scipy.ndimage import rotate
from scipy.interpolate import interp1d

# import h5py
from skimage.transform import radon
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import transforms
from PIL import Image, ImageFilter
import joblib as jb
import tifffile as tfl
import os
from IPython.display import clear_output
import re
import pandas as pd

from astropy.io import fits

In [ ]:
%run "C:\Users\gabri\Documents\Python Scripts\NPGMI-Library\NPGMI Base Code.ipynb"

In [ ]:
Lsg = 8
phisg = np.array([pi/2])
L1sg = 4

psg = 2.4e-6
# source_period = 200e-6

# duty_vals = 1/np.array([10,4,2])


polyspectrum = pd.read_excel("ng6_MCNP_spectrum.xlsx").to_numpy().T
polyspectrum[0]*=1e-9

# lamsg, plamsg = np.array([5e-10]), np.ones(1)
lamsg, plamsg = polyspectrum

mt = 5
x0 = np.array([0])



camsize = 1.5e-2



xbin = 1e-6
ybin = 1e-6
ressg = 100e-6
slitsg = 100e-6
dp2overlam = psg**2/lamsg[np.argmax(plamsg)]


# sgp_vals = Lsg * psg / dp2overlam / np.array([5,3,2,1, 1/0.9])
sgp_vals = np.array([1.5,1])*Lsg*lamsg[np.argmax(plamsg)]/psg

# sgp_vals = Lsg * psg / dp2overlam / np.array([5])
# sgp_vals = np.arange(1,5,1)*1e-4

dsg = np.linspace( 5, 18, 79)*1e-3


G1sdsg = {"p":psg, "L1": np.array([L1sg]), "phi":phisg}
G1sdsg["L2"] = dsg



G2sdsg = {"p":psg, "L1": G1sdsg["L1"] +  G1sdsg["L2"], "phi":phisg}
G2sdsg["L2"] = Lsg - (G1sdsg["L1"] + G1sdsg["L2"])



simaptssg = {"G1":RectGrating(G1sdsg),"G2":RectGrating(G2sdsg)}
siminitsg = {"lam":lamsg, "plam": plamsg, "L":Lsg,"d":G1sdsg["L2"],"mt":mt, "x": np.linspace(0,camsize,int(camsize/xbin) + 1),\
                 "x0" : x0,"xbin":xbin,"ybin":ybin,"pos":np.array([[0],[1]]) ,"camsize" : camsize ,"convmode" : "valid",\
                 "slitx":slitsg,"res": ressg, "batches" : 5000}


In [ ]:
t0 = time()

simsg = PGMI(simaptssg,siminitsg)

simsg.propagate_to("G1")
simsg.propagate_to("G2")
simsg.generate_after("G2")

contenvel = simsg.get_value("contrast")
plt.plot(contenvel)
plt.show()
print(time() - t0, "seconds")




In [ ]:
t0 = time()

duty_cycle = 0.5

contsgp50 = []
Isgp50 = []
fitparamssgp50 = []




for sgp in sgp_vals:
    siminitsg = {"lam":lamsg, "plam": plamsg, "L":Lsg,"d":G1sdsg["L2"],"mt":mt, "x": np.linspace(0,camsize,int(camsize/xbin) + 1),\
                 "x0" : x0,"xbin":xbin,"ybin":ybin,"pos":np.array([[0],[1]]) ,"camsize" : camsize ,"convmode" : "valid",\
                 "source_period":sgp, "duty_cycle":duty_cycle,"res": ressg, "batches" : 5000}
   
    print(sgp)
    print(duty_cycle)
    simsg = PGMI(simaptssg,siminitsg)
    simsg.propagate_to("G1")
    simsg.propagate_to("G2")
    simsg.generate_after("G2")
    
    contsgp50.append(simsg.get_value("contrast"))
    Isgp50.append(simsg.get_value("intensity"))
    fitparamssgp50.append(simsg.get_value("fitparams"))
    
    
    
print(time() - t0, "seconds")


In [ ]:
t0 = time()

duty_cycle = 0.1

contsgp10 = []
Isgp10 = []
fitparamssgp10 = []

for sgp in sgp_vals:
    siminitsg = {"lam":lamsg, "plam": plamsg, "L":Lsg,"d":G1sdsg["L2"],"mt":mt, "x": np.linspace(0,camsize,int(camsize/xbin) + 1),\
                 "x0" : x0,"xbin":xbin,"ybin":ybin,"pos":np.array([[0],[1]]) ,"camsize" : camsize ,"convmode" : "valid",\
                 "source_period":sgp, "duty_cycle":duty_cycle,"res": ressg, "batches" : 5000}
   
    print(sgp)
    print(duty_cycle)
    simsg = PGMI(simaptssg,siminitsg)
    simsg.propagate_to("G1")
    simsg.propagate_to("G2")
    simsg.generate_after("G2")
    
    contsgp10.append(simsg.get_value("contrast"))
    Isgp10.append(simsg.get_value("intensity"))
    fitparamssgp10.append(simsg.get_value("fitparams"))
    
print(time() - t0, "seconds")


In [ ]:
fig, ax1 = plt.subplots(figsize = (18,12))
dlamarr = np.array([2/3,1,4/3])*dp2overlam*1e3
dlamlabels = [r"$\frac{2p^{2}}{3\lambda_c}$", r"$\frac{p^{2}}{\lambda_c}$", r"$\frac{4p^{2}}{3\lambda_c}$" ]

ax1.set_prop_cycle(color = ["blue","blue","black"])

ax1.plot(simsg.d*1e3, contsgp50[0], lw = 2, label = r"DC = .5, $p_s = p_m \,[d = \frac{2p^{2}}{3\lambda_c}]$" % (sgp_vals[0] / (Lsg*psg/dp2overlam)) )
ax1.plot(simsg.d*1e3, contsgp10[0], "--", lw = 2, label = r"DC = .1, $p_s = p_m \,[d = \frac{2p^{2}}{3\lambda_c}]$" % (sgp_vals[0] / (Lsg*psg/dp2overlam)) )


ax1.set_prop_cycle(color = ["red","red","black"])
ax1.plot(simsg.d*1e3, contsgp50[1], lw = 2, label = r"DC = .5, $p_s = p_m \,[d = \frac{p^{2}}{\lambda_c}]$" % (sgp_vals[1] / (Lsg*psg/dp2overlam)) )

ax1.plot(simsg.d*1e3, contsgp10[1], "--", lw = 2, label = r"DC = .1, $p_s = p_m \,[d = \frac{p^{2}}{\lambda_c}]$" % (sgp_vals[1] / (Lsg*psg/dp2overlam)) )

ax1.plot(simsg.d*1e3, contenvel, lw = 2, label = r"$%.f \; \mu m$ slit" % (slitsg*1e6))


ax1.set_xlabel("Separation Distance $d$", fontsize = 30)
ax1.set_ylabel("Contrast", fontsize = 30)
ax1.legend(fontsize = 12)


[ax1.axvline(dlamarr[i], color = "black", ls = "dotted", label = dlamlabels[i]) for i in range(len(dlamarr))]

ax1.tick_params(axis = "x", labelsize = 30)
ax1.tick_params(axis = "y", labelsize = 20)

ax1.set_xticks(dlamarr)
ax1.set_xticklabels(dlamlabels)



# fig.savefig("SourceGratingPolychromaticContrast.png")
plt.show()
plt.close(fig)